# Workflow Example 1

This is based on a simple workflow, where only one cube is chosen with one dimension to plot over and a restricted number of customization options.

This is not a typical workflow, but it's a good start.

In [ ]:
import ipywidgets
import IPython.display
import iris

import glob
import numpy as np
import iris.quickplot as iplt
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

# 1. Choose and load your cube

Create cube picker consiting of a text acceptor for the file path and a multiple selection pane for the cube selector

Please enter the path of your cube(s) and then select the cube or cubelist you would like to plot with.

In [ ]:
path = ipywidgets.Text(
    description='Filepath',
    value='/tmp',
)
IPython.display.display(path)

In [ ]:
options = glob.glob('{}/*'.format(path.value)) 

files = ipywidgets.SelectMultiple(
    description='Cube(list)',
    options=options)

IPython.display.display(files)

In [ ]:
cubes = files.value

for i in range(len(cubes)):
    print cubes[i]
    if cubes[i][-2:] == 'nc' or cubes[i][-2:] == 'pp': # This is currently not taking grib files into 
                                                           # account coz they are hard to identify.  Also no ff.
        cubelist = iris.load_raw(cubes[i])
    else:
        print 'This is not a cube, please try another selection.'

print cubelist        

Select which cubes you would like to use if you have loaded a cubelist

In [ ]:
to_plot = [cube.standard_name for cube in cubelist]

plots = ipywidgets.SelectMultiple(
description='Choose cube(s)',
options=to_plot
)

IPython.display.display(plots)

# 2. Choose your plot options

Create plot-type picker

In [ ]:
plot_type_dict = {'contour': iplt.contour, 'contourf': iplt.contourf, 'pcolor': iplt.pcolor, 'outline': iplt.outline,
                  'pcolormesh': iplt.pcolormesh, 'plot': iplt.plot, 'points': iplt.points}

plot_types = plot_type_dict.keys()
plot_types.sort()

type = ipywidgets.Dropdown(
    options=plot_types,
    value='contour',
    description='Plot-type:')

IPython.display.display(type)

Create axis coordinate pickers for x and y axis

In [ ]:
base_cube = iris.load_cube(files.value, plots.value[0])
# base_cube is the first cube chosen from the list.
# This method will work provided that all the cubes chosen have the same coordinates.

coordinates = [(coord.name()) for coord in base_cube.coords()]
for i in range(len(plots.value)):
    coordinates.append(plots.value[i])

dim_x = ipywidgets.RadioButtons(
    description='Dimension for x:',
    options=coordinates)

IPython.display.display(dim_x)

In [ ]:
if dim_x.value in coordinates:
    coordinates.remove(dim_x.value)

dim_y = ipywidgets.RadioButtons(
    description='Dimension for y:',
    options=coordinates)

IPython.display.display(dim_y)

In [ ]:
if dim_y.value in coordinates:
    coordinates.remove(dim_y.value)

index = []
for i in range(len(coordinates)):
    coord_index = ipywidgets.IntSlider(
        value=0,
        min=0,
        max=len(base_cube.coord(coordinates[i]).points-1),
        step=1,
        description='Index of ' + coordinates[i])
    index.append(coord_index)
    IPython.display.display(index[i])

# 3. Choose your formatting options

In [ ]:
# All:
colors = ipywidgets.Dropdown(
    options=['None', 'blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'white'],
    value='blue',
    description='colors')
cmap = ipywidgets.Dropdown(
    options=['None', 'viridis', 'inferno', 'plasma', 'magma', 'jet', 'summer', 'autumn'],
    value='None',
    description='cmap')
norm = ipywidgets.Dropdown(
    options=['None', 'Autoscale(A)', 'Autoscale_None(A)', 'scaled()'],
    value='None',
    description='norm')
levels = ipywidgets.Text(
    description='levels',
    value='np.linspace(np.min(cube.data), np.max(cube.data), 10)')
origin = ipywidgets.Dropdown(
    options=['None', 'upper', 'lower', 'image'],
    value='None',
    description='origin')
extend = ipywidgets.Dropdown(
    options=['neither', 'both', 'min', 'max'],
    value='both',
    description='extend')
# Contour-only:
linestyles = ipywidgets.Dropdown(
    options=['None', 'solid', 'dashed', 'dashdot', 'dotted'],
    value='None',
    description='linestyle')
# Plot only:
label = ipywidgets.Text(
    description='label',
    value='Line Plot')
# Plot and Points only:
color = ipywidgets.Dropdown(
    options=['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'white'],
    value='blue',
    description='color')
marker = ipywidgets.Dropdown(
    options=['-', '--', '-.', ':', '.', ',', 'o', 'v', '^', '<', '>', '*', 'x'],
    value='-',
    description='marker')
    
formatting = [colors, cmap, norm, levels, origin, extend]
    
if type.value in ['contourf', 'outline', 'pcolor', 'pcolormesh']:
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
elif type.value == 'contour':
    formatting.append(linestyles)
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
elif type.value == 'plot':
    formatting.append(label)
    formatting.append(color)
    formatting.append(marker)
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
elif type.value == 'points':
    formatting.remove(colors)
    formatting.append(color)
    formatting.append(marker)    
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
    

# 4. Make your plot

This will require a button which composes your plot when you press it.  For simplicity, I have left this out of this workflow.

In [ ]:
callable = plot_type_dict[type.value]

plot_opts = {'colors':colors.value, 'cmap':cmap.value, 
                 'norm':norm.value, 'levels':eval(levels.value), 'origin':origin.value, 'extend':extend.value}
if type.value == 'contour':
    plot_opts['linestyles'] = linestyles.value
elif type.value == 'plot':
    plot_opts['label'] = label.value
    plot_opts['color'] = color.value
    plot_opts['marker'] = marker.value   
elif type.value == 'points':
    del plot_opts['colors']
    plot_opts['color'] = color.value
    plot_opts['marker'] = marker.value    

